In [1]:
import pygame
import random
import os
import numpy as np
import math

pygame.init()
pygame.mixer.init()

WIDTH = 1200
HEIGHT = 700
MaxDist = math.sqrt(WIDTH**2 + HEIGHT**2)
FPS = 30
car_acceleration = 2
handling = 0.1
speed_damping = 0.95
max_speed = 10
raycast_resolution = 5 #smaller is more accurate but slower
raycast_angle = 0.785

game_folder = os.getcwd()
#image_folder = os.path.join(game_folder,"img")

def convert_value_to_range(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

def rotateVector(vec,theta):
    rotationMatrix = [[math.cos(theta),-1*math.sin(theta)],
                    [math.sin(theta),math.cos(theta)]]
    rotationMatrix = np.array(rotationMatrix)
    np_vec = np.array(vec)
    new_vec = np.matmul(rotationMatrix,np_vec)

    return new_vec.tolist()

def rot_center_car(image, rect, angle):
    rot_image = pygame.transform.rotate(pygame.transform.scale(car_sprite,(40,20)), angle*180/(3.14))
    rot_image.set_colorkey((0,0,0))
    pos_x = rect.centerx
    pos_y = rect.centery
    rot_rect = rot_image.get_rect()
    rot_rect.centerx = pos_x
    rot_rect.centery = pos_y
    return rot_image,rot_rect

def rot_center(image, rect, angle):
    orig_img = pygame.Surface((1,MaxDist))
    orig_img.fill((255,0,0))
    orig_img.set_colorkey((0,0,0))
    rot_image = pygame.transform.rotate(orig_img, angle*180/(3.14))
    rot_image.set_colorkey((0,0,0))
    pos_x = rect.centerx
    pos_y = rect.centery
    rot_rect = rot_image.get_rect()
    rot_rect.centerx = pos_x
    rot_rect.centery = pos_y
    return rot_image,rot_rect

class Car(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.transform.scale(car_sprite,(40,20))
        self.image.set_colorkey((0,0,0))
        self.rect = self.image.get_rect()
        self.rect.x = WIDTH/2
        self.rect.y = HEIGHT/2
        self.speed = 0
        self.forward = [1,0]
        self.mask = pygame.mask.from_surface(self.image)
        self.rot = 0
        self.distTraveled = 0

    def update(self):
            self.speed *= speed_damping

            '''
            keystate = pygame.key.get_pressed()
            if keystate[pygame.K_LEFT]:
                self.forward = rotateVector(self.forward,-1*handling)
                self.rot += handling
                self.image,self.rect = rot_center_car(self.image,self.rect,self.rot)
                self.mask = pygame.mask.from_surface(self.image)
            if keystate[pygame.K_RIGHT]:
                self.forward = rotateVector(self.forward,handling)
                self.rot -= handling
                self.image,self.rect = rot_center_car(self.image,self.rect,self.rot)
                self.mask = pygame.mask.from_surface(self.image)
            if keystate[pygame.K_UP]:
                self.speed += car_acceleration
            if keystate[pygame.K_DOWN]:
                self.speed -= car_acceleration
            '''

            if self.speed>max_speed:
                self.speed = max_speed

            if self.speed < -1*max_speed: #test to see if this condition isn't satisfied if cars will drive backward
                self.speed = -1*max_speed
            self.rect.x += self.speed * self.forward[0]
            self.rect.y += self.speed * self.forward[1]
            self.distTraveled += self.speed
            if self.rect.right > WIDTH:
                self.rect.right = WIDTH
            if self.rect.left < 0:
                self.rect.left = 0
    def accelerate(self,value):
        value = convert_value_to_range(value,0,1,-1*car_acceleration,car_acceleration)
        self.speed += value
    
    def steer(self,value):
        value = convert_value_to_range(value,0,1,-1*handling,handling)
        self.forward = rotateVector(self.forward,value)
        self.rot += value
        self.image,self.rect = rot_center_car(self.image,self.rect,self.rot)
        self.mask = pygame.mask.from_surface(self.image)

    def get_raycast(self,type,trackGroup):
        forward_vec = None
        if type is "front":
            forward_vec = self.forward
        elif type is "right":
            forward_vec = rotateVector(self.forward,raycast_angle)
        elif type is "left":
            forward_vec = rotateVector(self.forward,-1*raycast_angle)
        totalDist = 0
        speedx = raycast_resolution*forward_vec[0]
        speedy = raycast_resolution*forward_vec[1]
        moving_point = RaycastPoint(self.rect.centerx,self.rect.centery)
        while totalDist<MaxDist:
            #check for collisions
            hits = pygame.sprite.spritecollide(moving_point,trackGroup,False,pygame.sprite.collide_mask)
            if len(hits) > 0:
                return math.sqrt((moving_point.rect.x-self.rect.centerx)**2 + (moving_point.rect.y-self.rect.centery)**2)
            #update point position
            moving_point.update(speedx,speedy)
            totalDist += raycast_resolution


        return "no collisions"


class Track(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = track_1
        self.image.set_colorkey((0,0,0))
        self.rect = self.image.get_rect()
        self.rect.x = 0
        self.rect.y = 0
        self.mask = pygame.mask.from_surface(self.image)

    def update(self):
        pass

class RaycastPoint(pygame.sprite.Sprite):
    def __init__(self,x_init,y_init):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((10,10))
        self.image.fill((255,0,0))
        self.image.set_colorkey((0,0,0))
        self.rect = self.image.get_rect()
        self.rect.x = x_init
        self.rect.y = y_init
        self.mask = pygame.mask.from_surface(self.image)

    def update(self,speed_x,speed_y):
        self.rect.x += speed_x
        self.rect.y += speed_y
        self.mask = pygame.mask.from_surface(self.image)


screen = pygame.display.set_mode((WIDTH,HEIGHT))
track_1 = pygame.image.load(os.path.join(game_folder,"track_1.png")).convert()
car_sprite = pygame.image.load(os.path.join(game_folder,"car.png")).convert()



class env():
    def __init__(self):
        self.all_sprites = pygame.sprite.Group()
        self.tracks = pygame.sprite.Group()
        self.player = Car()
        collision_test = Track()

        self.all_sprites.add(self.player)
        self.all_sprites.add(collision_test)
        self.tracks.add(collision_test)
        self.score = 0
    
    def step(self,actions):
        self.player.accelerate(actions[0])
        self.player.steer(actions[1])
        self.all_sprites.update()
        game_state = self.state()
        
        observations = self.get_observations()
        
        if game_state:
            self.score += 1
        
        return observations,game_state,self.player.distTraveled
    
    def get_observations(self):
        observations = []
        results = self.player.get_raycast("front",self.tracks)
        if not (results=="no collisions"):
            observations.append(results)
        results = self.player.get_raycast("right",self.tracks)
        if not (results=="no collisions"):
            observations.append(results)
        results = self.player.get_raycast("left",self.tracks)
        if not (results=="no collisions"):
            observations.append(results)
        
        if len(observations) < 3:
            print("Less than 3 observations!!!")
        
        return observations
    
    def state(self):
        hits = pygame.sprite.spritecollide(self.player,self.tracks,False,pygame.sprite.collide_mask)
        if len(hits) > 0:
            return False
        return True

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def relu(x):
    if x<0:
        return 0
    else:
        return x

def sigmoid(x):
    return 1/(math.exp(-1*x)+1)
def make_decision_manual(observations,parameters):
    in_h1 = np.array(parameters[:12]).reshape((3,4))
    h1_bias = np.array(parameters[12:16]).reshape((1,4))
    h1_out = np.array(parameters[16:24]).reshape((4,2))
    out_bias = np.array(parameters[24:26]).reshape((1,2))
    input_layer = np.array(observations).reshape((1,3))
    h1_layer = (np.matmul(input_layer,in_h1)+h1_bias)
    for i in range(0,4):
        h1_layer[0][i] = sigmoid(h1_layer[0][i])
    output_layer = np.array(list(np.matmul(h1_layer,h1_out)+out_bias)).reshape((1,2))
    for i in range(2):
        output_layer[0][i] = sigmoid(output_layer[0][i])
    actions = [output_layer[0][0],output_layer[0][1]]
    return actions

def map_observations(observations):
    observations[2] = convert_value_to_range(observations[2],0,MaxDist,0,5)
    observations[1] = convert_value_to_range(observations[1],0,MaxDist,0,5)
    observations[0] = convert_value_to_range(observations[0],0,MaxDist,0,5)
    return observations
    
def take_decision(observations,parameters):
    observations = map_observations(observations)
    answer = make_decision_manual(observations,parameters)
    return answer

In [3]:
def run_episode(parameters):
    game_env = env()
    observations = game_env.get_observations()
    total_reward = 0
    for _ in range(1000):
        action = take_decision(observations,parameters)
        observations,alive,score = game_env.step(action)
        total_reward = score
        if not alive:
            break
    return total_reward

def run_episodes(n,parameters):
    total_reward = 0
    for _ in range(n):
        total_reward += run_episode(parameters)
    return total_reward/n

def normalize(arr):
    return arr / np.linalg.norm(arr)

def eval_function(parameters):
    return run_episodes(sampling_number,parameters)

In [4]:
sampling_number = 3
value = eval_function(np.random.uniform(-1,1,26))
#pygame.quit()

In [ ]:
import random
from deap import creator, base, tools, algorithms

'''
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=4)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_func(individual):
    #return sum(individual)
    return run_episodes(sampling_number,individual),

toolbox.register("evaluate", eval_func)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0.0, sigma=0.2, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=500)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
    top_player = tools.selBest(population,k=1)[0]
    print("Generation number {}, best has fitness {}".format(gen+1,run_episodes(sampling_number,top_player)))
'''

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=26)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_func(individual):
    #return sum(individual)
    return run_episodes(sampling_number,individual),

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", eval_func)

population = toolbox.population(n=500)

NGEN = 200
CXPB = 0.8
MUTPB = 0.05

best_player = None
best_player_score = 0
for gen in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    population[:] = offspring
    
    top_player = tools.selBest(population,k=1)[0]
    top_player_score = run_episodes(sampling_number,top_player)
    if top_player_score > best_player_score:
        best_player = top_player
        best_player_score = top_player_score
    print("Generation number {}, best has fitness {}".format(gen+1,top_player_score))
top10 = tools.selBest(population, k=10)

C:\Users\Kareem Elsawah\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


Generation number 1, best has fitness 97.65959198389402
Generation number 2, best has fitness 109.4984595756477
Generation number 3, best has fitness 112.70303484633813
Generation number 4, best has fitness 109.52587435988028
Generation number 5, best has fitness 109.71967186273825
Generation number 6, best has fitness 111.21451088523374
Generation number 7, best has fitness 116.4018943345964
Generation number 8, best has fitness 116.00259359612562
Generation number 9, best has fitness 110.83784231411806
Generation number 10, best has fitness 110.83784231411806
Generation number 11, best has fitness 110.83784231411806
Generation number 12, best has fitness 112.54844739092145
Generation number 13, best has fitness 113.40219436119526
Generation number 14, best has fitness 113.40219436119526
Generation number 15, best has fitness 110.47092824456172
Generation number 16, best has fitness 112.14948342451288
Generation number 17, best has fitness 112.22470377168709
Generation number 18, best

In [ ]:
NGEN = 200
CXPB = 0.8
MUTPB = 0.05

#best_player = None
#best_player_score = 0
for gen in range(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(population, len(population))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    population[:] = offspring
    
    top_player = tools.selBest(population,k=1)[0]
    top_player_score = run_episodes(sampling_number,top_player)
    if top_player_score > best_player_score:
        best_player = top_player
        best_player_score = top_player_score
    print("Generation number {}, best has fitness {}".format(gen+1,top_player_score))
top10 = tools.selBest(population, k=10)

Generation number 1, best has fitness 242.43859384774592
Generation number 2, best has fitness 242.43859384774592
Generation number 3, best has fitness 242.43859384774592
Generation number 4, best has fitness 242.43859384774592
Generation number 5, best has fitness 242.43859384774592
Generation number 6, best has fitness 242.43859384774592
Generation number 7, best has fitness 242.43859384774592
Generation number 8, best has fitness 242.43859384774592
Generation number 9, best has fitness 242.43859384774592
Generation number 10, best has fitness 242.43859384774592
Generation number 11, best has fitness 242.43859384774592
Generation number 12, best has fitness 242.43859384774592


In [ ]:
pygame.quit()